In [1]:
import sys
sys.path.append('/home/trukhinmaksim/src')
import json

In [2]:
from time import time
from src.utils.CacheAdapter import FlatAdapter, EXP_END_OF_DATA, createAdapter_02_04_25_GOOD, createTrainSetAdapter_02_04_25_GOOD, createTestSetAdapter_02_04_25_GOOD, createNormAdapter_02_04_25_GOOD
from src.utils.Corpus import CacheCorpus, FlatCorpus
from src.utils.DatasetManager import ProjectsDatasetManager

In [3]:
def clearFiles():
    with open("/home/trukhinmaksim/src/data/train_02-04-25/train_02-04-25", "w") as file:
        pass
    with open("/home/trukhinmaksim/src/data/train_02-04-25/test_02-04-25", "w") as file:
        pass
#clearFiles()

In [4]:
def createMainCorpus():
    adapter = createNormAdapter_02_04_25_GOOD()#JSONMultiFileAdapter(CACHE_FILE_NAME)
    #manager = ProjectsDatasetManager(50, cacheAdapter = adapter)
    return FlatCorpus(adapter)

trainsetAdapter = createTrainSetAdapter_02_04_25_GOOD()
testsetAdapter = createTestSetAdapter_02_04_25_GOOD()

In [5]:
DATASET_SIZE = 482555

testsetSize = int(DATASET_SIZE * .15)
trainsetSize = int(DATASET_SIZE * .85)

testsetSize

74980

In [6]:
corpus = createMainCorpus()

def showTagsDistrubutionForCorpus(corpus):
    tagsCount = {}
    tagsLst = []

    def sort(tagsCount):
        print("Sorting tags")
        tagsLst = sorted([*tagsCount.items()], key = lambda pair_1: pair_1[1], reverse = True)

        return tagsLst


    start = time()
    i = 0
    try:
        for proj in corpus:
            if i % 100000 == 0: 
                print(f"Scanned {i} projects in {time() - start} s")
            for tag in proj.tags:
                if tag in tagsCount:
                    tagsCount[tag] += 1
                else:
                    tagsCount[tag] = 1

            i += 1
    except Exception as exp:
        raise exp
    finally:
        tagsLst = sort(tagsCount)
        tagsCount.clear()
    
        return tagsLst

tagsLst = showTagsDistrubutionForCorpus(corpus)

working_list = tagsLst[:120] # 116

working_list

Scanned 0 projects in 0.0002620220184326172 s
Scanned 10000 projects in 0.10823798179626465 s
Scanned 20000 projects in 0.1796708106994629 s
Scanned 30000 projects in 0.25379204750061035 s
Scanned 40000 projects in 0.325608491897583 s
Scanned 50000 projects in 0.3895750045776367 s
Scanned 60000 projects in 0.4549596309661865 s
Scanned 70000 projects in 0.519320011138916 s
Scanned 80000 projects in 0.5994408130645752 s
Scanned 90000 projects in 0.6653387546539307 s
Scanned 100000 projects in 0.7265615463256836 s
Scanned 110000 projects in 0.7923059463500977 s
Scanned 120000 projects in 0.8639407157897949 s
Scanned 130000 projects in 0.930325984954834 s
Scanned 140000 projects in 0.9988327026367188 s
Scanned 150000 projects in 1.071631908416748 s
Scanned 160000 projects in 1.145146131515503 s
Scanned 170000 projects in 1.2411198616027832 s
Scanned 180000 projects in 1.3094537258148193 s
Scanned 190000 projects in 1.3780715465545654 s
Scanned 200000 projects in 1.442676305770874 s
Scanned

[('JavaScript', 88961),
 ('Python', 67907),
 ('TypeScript', 49225),
 ('C++', 40028),
 ('Java', 30200),
 ('HTML', 24079),
 ('C', 22677),
 ('Go', 18000),
 ('Jupyter Notebook', 15796),
 ('Shell', 14293),
 ('C#', 13189),
 ('PHP', 11866),
 ('javascript', 11358),
 ('react', 10563),
 ('python', 10446),
 ('Rust', 9016),
 ('hacktoberfest', 8918),
 ('CSS', 8854),
 ('typescript', 8218),
 ('Kotlin', 7335),
 ('nodejs', 7208),
 ('Ruby', 5708),
 ('Vue', 5416),
 ('Swift', 4445),
 ('java', 4427),
 ('config', 3981),
 ('reactjs', 3930),
 ('Dart', 3833),
 ('css', 3710),
 ('github-config', 3686),
 ('android', 3607),
 ('docker', 3391),
 ('Dockerfile', 3154),
 ('html', 3149),
 ('cpp', 3094),
 ('golang', 3059),
 ('Lua', 2928),
 ('python3', 2863),
 ('php', 2659),
 ('machine-learning', 2587),
 ('api', 2580),
 ('linux', 2541),
 ('vue', 2453),
 ('nextjs', 2418),
 ('node', 2394),
 ('SCSS', 2361),
 ('cli', 2350),
 ('Nix', 2332),
 ('rust', 2226),
 ('go', 2158),
 ('deep-learning', 2146),
 ('c', 1983),
 ('react-native

In [7]:
def countProjectsWithTagsToRemove(corpus):
    counter = 0
    tagsNamesToRemove = set((p[0] for p in working_list))

    for doc in corpus:
        tags = tagsNamesToRemove & set(doc.tags[2:])

        if len(tags): counter += 1

    return counter

total_working_proj_with_tags = countProjectsWithTagsToRemove(corpus)
total_working_proj_with_tags

474025

In [8]:
from math import ceil
tagsToRemove = {}

k = 1.17 # special parameter, must be adjusted to collect right amount of documents into the testing set 
for item in working_list:
    pair = list(item)
    pair[1] = ceil(k * pair[1] / total_working_proj_with_tags * testsetSize)

    tagsToRemove[pair[0]] = pair[1]

tagsNamesToRemove = set(tagsToRemove.keys())

[*tagsToRemove.items()][-5:]

[('vite', 181),
 ('angular', 179),
 ('frontend', 176),
 ('Perl', 175),
 ('Vim script', 175)]

In [9]:
def decreaseTags(tagsToRemove, tags):
    for tag in tags:
        tagsToRemove[tag] -= 1

def keepDoc(tagsToRemove, tags): 
    for tag in tags:
        if tagsToRemove[tag] == 0:
            return True

    # if all tags can be removed, then False
    return False

def splitSets(corpus):
    ignoreDocs = set()
    testSetCounter = 0
    i = 0

    for doc in corpus:
        tags = tagsNamesToRemove & set(doc.tags[1:]) # tags[0] is an id, shouldn't be viewed

        if len(tags):
            # if project contains tags, that are in 'working list'

            if not keepDoc(tagsToRemove, tags):
                # write to the testing set
                testsetAdapter.save([{"tokens" : doc.words, "tags" : doc.tags}])
                ignoreDocs.add(doc.tags[0])

                if len(ignoreDocs) >= testsetSize:
                    print(f"Collected {len(ignoreDocs)} on iteration {i}, test set updated")
                    break
                decreaseTags(tagsToRemove, tags)
        i += 1

    corpus.reset()
    #for doc in corpus: pass # to exhaust the corpus

    counter = 0
    for doc in corpus:
        if doc.tags[0] not in ignoreDocs:
            trainsetAdapter.save([{"tokens" : doc.words, "tags" : doc.tags}])
            counter += 1

    print(f"Train set updated with {counter} documents")

splitSets(corpus)

Collected 74980 on iteration 99021


In [ ]:
# TODO: finish that class for corpus separation
class CorpusSeparator:
    class EXP_PROPORTION_WRONG(Exception):
        def __init__(self):
            super().__init__("Wrong proportion specified, train and test sets sizes portions must add up to 100")
    
    def __init__(self, mainCorpus, trainAdapter, testAdapter, trainPortion = 85, testPortion = 15):
        # will take all data from mainCorpus and separate it into train and test set, according to proportions
        # algorithm = "balanced" | "random" | "first n"
        #     "balanced" will automatically determine, how many project with each tag should be moved into the test set, more projects with popular tags will be moved to test set, less projects with unpopular tags will be moved

        if trainPortion + testPortion != 100: raise EXP_PROPORTION_WRONG

        corpusLen = 0
        for doc in mainCorpus:
            corpusLen += 1

        testsetSize = int(corpusLen * testPortion / 100)
        trainsetSize = int(corpusLen * trainPortion / 100)


    def balanced(self):
        tagsCount = {}
        tagsLst = []

        def sort(tagsCount):
            print("Sorting tags")
            tagsLst = sorted([*tagsCount.items()], key = lambda pair_1: pair_1[1], reverse = True)

            return tagsLst

        start = time()
        i = 0
        try:
            for proj in corpus:
                if i % 100000 == 0: 
                    print(f"Scanned {i} projects in {time() - start} s")
                for tag in proj.tags:
                    if tag in tagsCount:
                        tagsCount[tag] += 1
                    else:
                        tagsCount[tag] = 1

                i += 1
        except Exception as exp:
            raise exp
        finally:
            tagsLst = sort(tagsCount)
            tagsCount.clear()

            working_list = tagsLst[:120]

        counter = 0
        tagsNamesToRemove = set((p[0] for p in working_list))

        for doc in corpus:
            tags = tagsNamesToRemove & set(doc.tags[2:])

            if len(tags): counter += 1

        return counter

        total_working_proj_with_tags = countProjectsWithTagsToRemove(corpus)
    
    def separate(self, algorithm = "balanced"):
        pass

In [10]:
def checkCorpusesSizes():
    trainCorpus = FlatCorpus(trainsetAdapter)
    testCorpus = FlatCorpus(testsetAdapter)

    for corp, name in ((trainCorpus, "Train"), (testCorpus, "Test")):
        c = 0
        for doc in corp:
            c += 1
        print(f"{name} contains {c}")

Test contains 74980
Train contains 424888


In [14]:
#showTagsDistrubutionForCorpus(trainCorpus)[:50]

Scanned 0 projects in 6.628036499023438e-05 s
Scanned 10000 projects in 0.10113835334777832 s
Scanned 20000 projects in 0.16685962677001953 s
Scanned 30000 projects in 0.22852802276611328 s
Scanned 40000 projects in 0.29355382919311523 s
Scanned 50000 projects in 0.35941481590270996 s
Scanned 60000 projects in 0.42553210258483887 s
Scanned 70000 projects in 0.49033617973327637 s
Scanned 80000 projects in 0.5597727298736572 s
Scanned 90000 projects in 0.6201627254486084 s
Scanned 100000 projects in 0.6784758567810059 s
Scanned 110000 projects in 0.7380290031433105 s
Scanned 120000 projects in 0.8009552955627441 s
Scanned 130000 projects in 0.8603661060333252 s
Scanned 140000 projects in 0.9204046726226807 s
Scanned 150000 projects in 0.9820616245269775 s
Scanned 160000 projects in 1.0500364303588867 s
Scanned 170000 projects in 1.134218454360962 s
Scanned 180000 projects in 1.1960618495941162 s
Scanned 190000 projects in 1.2591915130615234 s
Scanned 200000 projects in 1.3235540390014648

[('JavaScript', 72496),
 ('Python', 59662),
 ('TypeScript', 40111),
 ('C++', 34037),
 ('Java', 26345),
 ('HTML', 19621),
 ('C', 18478),
 ('Go', 15118),
 ('Jupyter Notebook', 14835),
 ('Shell', 11819),
 ('C#', 11277),
 ('PHP', 9668),
 ('javascript', 9247),
 ('python', 8635),
 ('react', 8597),
 ('Rust', 7530),
 ('hacktoberfest', 7265),
 ('CSS', 7215),
 ('typescript', 6695),
 ('Kotlin', 6219),
 ('nodejs', 5868),
 ('Ruby', 4651),
 ('Vue', 4413),
 ('java', 3758),
 ('Swift', 3695),
 ('config', 3459),
 ('Dart', 3268),
 ('github-config', 3231),
 ('reactjs', 3202),
 ('css', 3018),
 ('android', 2929),
 ('docker', 2764),
 ('Dockerfile', 2648),
 ('cpp', 2607),
 ('html', 2563),
 ('Lua', 2498),
 ('golang', 2489),
 ('python3', 2431),
 ('machine-learning', 2292),
 ('Nix', 2247),
 ('php', 2166),
 ('node', 2120),
 ('api', 2084),
 ('linux', 2081),
 ('nextjs', 1968),
 ('vue', 1966),
 ('deep-learning', 1955),
 ('SCSS', 1924),
 ('cli', 1877),
 ('rust', 1813)]